# Адаптер (adapter)

Адаптер — это структурный паттерн проектирования, который позволяет объектам с несовместимыми интерфейсами работать вместе.

## Пример

При разработке сервиса авторизации, работа была декомпозирована 2 отделам. В тех. задании была допущена
ошибка, из-за которой одна команда в качестве идентификатора клиента использовала `UUID`, а другая — `Microsoft LDAP ObjectGuid`. Требуется состыковать код, унифицировав аутентификацию к `UUID` виду.


In [37]:
import abc


class LDAPGuidAuth:
    """
    Класс, который реализовала первая команда.
    """

    id = ""

    def auth(self, login: str, password: str) -> None:
        # Для демонстрационных целей захардкодил конкретный `LDAP ObjectGuid`
        self.id = "x+QSIh4FDE2aWxJ3Cpu3qw=="

    @property
    def ldap_id(self) -> str:
        return self.id


class AbsUUIDAuth:
    """
    В дальнейшем, приведем адаптируемый класс к данному "интерфейсу"
    """

    id = ""

    @abc.abstractproperty
    def uuid_id(self) -> str:
        ...


class UUIDAuth(AbsUUIDAuth):
    """
    Класс, который реализовала вторая команда
    """

    def auth(self, login: str, password: str) -> None:
        # Для демонстрационных целей захардкодил конкретный `Python UUID`
        self.id = "2212e4c7-051e-4d0c-9a5b-12770a9bb7ab"

    @property
    def uuid_id(self) -> str:
        return self.id

In [38]:
import base64
import uuid
from typing import Any


class AbsAdapter(abc.ABC):
    """
    Класс адаптера, которому передам в качестве параметра исходный объект (адаптируемый)
    Реализация через композицию
    """

    def __init__(self, source: Any) -> None:
        self._source_obj = source


class LDAPGuidAdapter(AbsAdapter, AbsUUIDAuth):
    """
    Реализация адаптера, для использования `LDAPGuidAuth` там, где ожидается `UUIDAuth`
    """

    @property
    def uuid_id(self) -> str:
        """
        Преобразуем `id` вида `LDAPGuidAuth` в `UUID`
        """
        return str(uuid.UUID(bytes_le=base64.b64decode(self._source_obj.id)))

In [39]:
ldap_base_auth = LDAPGuidAuth()
uuid_base_auth = UUIDAuth()

ldap_base_auth.auth("user", "example")
uuid_base_auth.auth("user", "example")

# Преобразовываем к общему интерфейсу, если не сделаем, получим `AttributeError`
adapted_ldap_auth = LDAPGuidAdapter(ldap_base_auth)

# У исходного класса `LDAPGuidAuth` не было этого свойства
assert uuid_base_auth.uuid_id == adapted_ldap_auth.uuid_id